In [1]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [3]:
import json
with open('./bigFreqList.json') as json_file:
     wordsData = json.load(json_file)

In [4]:
#generate a list of words with frequency higher than the threshold
def generateWordList(threshold): 
  temp = []
  for (word, freq) in wordsData.items():
    if (freq >= threshold):
      temp.append(word)
  return temp

In [5]:
wordList = generateWordList(40)
print(wordList[-10:])
len(wordList)

['latikivn', 'imusicvn', 'ycđt', 'greatland', 'lemmevn', 'hoclamgiau', 'hpschoolvn', 'khochiasevn', 'hakiba', 'heisvn']


141836

In [6]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [7]:
for batch in batches:
  print(len(batch))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1836


In [8]:
posDict = {} #a dictionary to translate POS words in Vietnamese to English
posDict["Động từ"] = "verb"
posDict["Danh từ"] = "noun"
posDict["Nghĩa chuyên ngành"] = "jargon"
posDict["*"] = "N/A"
posDict["Trạng ngữ"] = "adverb"
posDict["Cảm thán"] = "interjection"
posDict["Phó từ"] = "adverb"
posDict["Từ nối"] = "conjunction"
posDict["Từ đệm"] = "N/A"
posDict["Đại từ"] = "pronoun"

In [9]:
#The link format for soha Viet-Eng is: http://tratu.soha.vn/dict/vn_en/C%C3%B3

In [10]:
url = "http://tratu.soha.vn/dict/vn_en/" + quote("như") #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [11]:
groups = soup.findAll("div", class_ = "section-h3") #soha groups their translations by POS

In [12]:
containers = soup.findAll("div", class_ = "section-h5")

In [13]:
containers[0].parent

<div class="section-h2" id="show-alter"><h2> <span class="mw-headline">Thông dụng</span></h2>
<a name="As."></a><div class="section-h5" id="content-5"><h5> <span class="mw-headline">As.</span></h5>
<dl><dd><dl><dd><a href="/dict/vn_en/L%C3%A0m" title="Làm">làm</a> <strong class="selflink">như</strong> <a href="/dict/vn_en/T%C3%B4i" title="Tôi">tôi</a>
</dd><dd>Do as I do.
</dd></dl>
</dd></dl>
<a name="Like."></a></div><div class="section-h5" id="content-5"><h5> <span class="mw-headline">Like.</span></h5>
<dl><dd><dl><dd><a href="/dict/vn_en/T%C3%B4i" title="Tôi">tôi</a> <a href="/dict/vn_en/C%C5%A9ng" title="Cũng">cũng</a> <a href="/dict/vn_en/Ngh%C4%A9" title="Nghĩ">nghĩ</a> <strong class="selflink">như</strong> <a href="/dict/vn_en/Anh" title="Anh">anh</a>
</dd><dd>I think like you.
</dd><dd>_alike; similar to.
</dd><dd><a href="/dict/vn_en/Tr%C6%B0%E1%BB%9Dng" title="Trường">trường</a> <a href="/dict/vn_en/H%E1%BB%A3p" title="Hợp">hợp</a> <a href="/dict/vn_en/Anh" title="Anh">anh</

In [14]:
containers[0].find("dl").find("dd").find("dl").findAll("dd")

[<dd><a href="/dict/vn_en/L%C3%A0m" title="Làm">làm</a> <strong class="selflink">như</strong> <a href="/dict/vn_en/T%C3%B4i" title="Tôi">tôi</a>
 </dd>,
 <dd>Do as I do.
 </dd>]

In [15]:
#detect whether a tag contains a link, if a link is present, the tag contains a context
def hasLink(tag):
    try:
        subTags = tag.findAll('a')
    except:
        return False
    for subTag in subTags:
        if (subTag.get('href')): #found a link
            return True
    return False

In [16]:
hasLink(containers[0].find("dl").find("dd").find("dl").findAll("dd")[1])

False

In [17]:
def extractContext(tag):
    parts = list(tag.children)
    parts = filter(lambda x: False if x == ' ' or x == '\n' else True, parts) #filter all the non-words out
    parts = [part.contents[0] for part in parts]
    context = " ".join(parts)
    return context

In [18]:
extractContext(containers[0].find("dl").find("dd").find("dl").findAll("dd")[0])

'làm như tôi'

In [19]:
unknownPos = []
def getPos(vietPos): #translating the POS in Viet to English
    vietPos = vietPos.strip(".")
    if (vietPos.find("(") != -1): #the vietPos contains the english equivalent
        return vietPos[vietPos.find("(")+1:vietPos.find(")")].strip()
    elif (vietPos in posDict):
        return posDict[vietPos]
    else:
        unknownPos.append(str(vietPos))
        return "N/A"

In [20]:
getPos("Động từ")

'verb'

In [21]:
getPos("Phó từ ( affirmative particle)")

'affirmative particle'

In [22]:
def resolveItalic(parts): #soha use italic to mark an use of "'"
    parts = [str(part) for part in parts]
    if (len(parts) == 1): #nothing special here, just return the string
        return [item.strip() for item in re.split('[.,;]', str(parts[0]))]
    parts = [part.replace("<i>", "'").replace("</i>", "") for part in parts]
    if (parts[-1] == '\n'): #this is an usage, ignore the last part
        return [item.strip() for item in re.split('[.,;]', "".join(parts[:-1]))]
    #this is a translation
    return [item.strip() for item in re.split('[.,;]', "".join(parts))]

In [23]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "http://tratu.soha.vn/dict/vn_en/" + quote(word) #resolving the IRI issue

    try:
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"

    soup = BeautifulSoup(source, 'lxml')
    
    translations = {}
    numTrans = 0
 
    containers = soup.findAll("div", class_ = "section-h5") #getting all the translations for a word
    if (containers is None or containers == []):
      print("No translations found for " + word)
      return "N/A"
 
    for trans in containers: #for each translation in this group
        temp = {} #temp dict to store each translation of a word

        #getting the raw translation
        temp['translation'] = resolveItalic(trans.find("h5").find("span").contents)
        
        parentTag = trans.parent
        try:
          if (parentTag.attrs["class"] == "section-h3"): #has POS
            temp['POS'] = getPos(group.find("h3").find("span").string)
          else:
            temp['POS'] = "N/A"
        except: #no POS
            temp['POS'] = "N/A"

        try:
            #retrieving the examples and contexts of this translation
            clarifications = trans.find("dl").find("dd").find("dl").findAll("dd")
            temp2 = {} #another dict to store each examples of a translation
            
            numContext = 0;
            numUsage = 0;
            
            for item in clarifications:
                if (hasLink(item)): #this is a context (context contains hyperlink)
                    temp2['context ' + str(numContext+1)] = extractContext(item)
                    numContext += 1
                else: #this is an usage
                    temp2['usage ' + str(numUsage+1)] = resolveItalic(item.contents)
                    numUsage += 1
            
            temp2['nums context'] = numContext
            temp2['nums usage'] = numUsage

            temp['examples'] = temp2
        except:
            temp['examples'] = "N/A"

        translations[numTrans+1] = temp #adding the translation to the dictionary
        numTrans += 1
            
    #storing the number of translations
    translations['nums'] = numTrans
    return translations        

In [24]:
findTranslation("như")

Finding translation for như


{1: {'translation': ['As', ''],
  'POS': 'N/A',
  'examples': {'context 1': 'làm như tôi',
   'usage 1': ['Do as I do', ''],
   'nums context': 1,
   'nums usage': 1}},
 2: {'translation': ['Like', ''],
  'POS': 'N/A',
  'examples': {'context 1': 'tôi cũng nghĩ như anh',
   'usage 1': ['I think like you', ''],
   'usage 2': ['_alike', 'similar to', ''],
   'context 2': 'trường hợp anh giống như trường hợp tôi',
   'usage 3': ['Your case is similar to mine', ''],
   'nums context': 2,
   'nums usage': 3}},
 'nums': 2}

In [25]:
findTranslation("tôi")

Finding translation for tôi


{1: {'translation': ['Danh từ'],
  'POS': 'N/A',
  'examples': {'usage 1': ['subject'],
   'usage 2': ['servant'],
   'usage 3': ['self'],
   'nums context': 0,
   'nums usage': 3}},
 2: {'translation': ['Động từ'],
  'POS': 'N/A',
  'examples': {'usage 1': ['I', 'me'],
   'usage 2': ['to temper', 'to slake'],
   'nums context': 0,
   'nums usage': 2}},
 3: {'translation': ['bad'], 'POS': 'N/A', 'examples': 'N/A'},
 4: {'translation': ['friable'], 'POS': 'N/A', 'examples': 'N/A'},
 5: {'translation': ['chilling'], 'POS': 'N/A', 'examples': 'N/A'},
 6: {'translation': ['cure'], 'POS': 'N/A', 'examples': 'N/A'},
 7: {'translation': ['dip'], 'POS': 'N/A', 'examples': 'N/A'},
 8: {'translation': ['thoriveitite'], 'POS': 'N/A', 'examples': 'N/A'},
 9: {'translation': ['advent'], 'POS': 'N/A', 'examples': 'N/A'},
 10: {'translation': ['arrival'], 'POS': 'N/A', 'examples': 'N/A'},
 11: {'translation': ['arrive'], 'POS': 'N/A', 'examples': 'N/A'},
 12: {'translation': ['ingoing'], 'POS': 'N/A'

In [26]:
#from google.colab import files
start_time = time.time()

index = 3
batch = batches[index] + batches[index+1]

dictionary = {}
unknownPos = []

#making the dictionary for each batch
for word in batch:
    dictionary[word] = findTranslation(word)
    time.sleep(1) #wait for 1 sec between each query

dictionary['unknownPos'] = unknownPos

Finding translation for dany
No translations found for dany
Finding translation for dane
No translations found for dane
Finding translation for daejeon
No translations found for daejeon
Finding translation for dz
No translations found for dz
Finding translation for dmm
No translations found for dmm
Finding translation for dhq
No translations found for dhq
Finding translation for dh04
No translations found for dh04
Finding translation for d20
No translations found for d20
Finding translation for d001
No translations found for d001
Finding translation for crete
No translations found for crete
Finding translation for crafted
No translations found for crafted
Finding translation for conversion
No translations found for conversion
Finding translation for condoleezza
No translations found for condoleezza
Finding translation for cienco4
No translations found for cienco4
Finding translation for chôl
No translations found for chôl
Finding translation for chnăm
No translations found for chnăm
Fi

No translations found for xổng
Finding translation for xằng
No translations found for xằng
Finding translation for xtraseal
No translations found for xtraseal
Finding translation for websitechúng
No translations found for websitechúng
Finding translation for vndm2
No translations found for vndm2
Finding translation for validate
No translations found for validate
Finding translation for undertone
No translations found for undertone
Finding translation for tửcân
No translations found for tửcân
Finding translation for tấntháng
No translations found for tấntháng
Finding translation for têu
No translations found for têu
Finding translation for têm
Finding translation for tâm1
No translations found for tâm1
Finding translation for trằng
No translations found for trằng
Finding translation for trạn
No translations found for trạn
Finding translation for tróng
No translations found for tróng
Finding translation for triệungày
No translations found for triệungày
Finding translation for transducer


No translations found for vln
Finding translation for viv
No translations found for viv
Finding translation for vitours
No translations found for vitours
Finding translation for vca
No translations found for vca
Finding translation for usdxe
No translations found for usdxe
Finding translation for ucl
No translations found for ucl
Finding translation for ubqg
No translations found for ubqg
Finding translation for u14
No translations found for u14
Finding translation for tuanzebe
No translations found for tuanzebe
Finding translation for trophy
No translations found for trophy
Finding translation for tribulus
No translations found for tribulus
Finding translation for tresemme
No translations found for tresemme
Finding translation for treponema
No translations found for treponema
Finding translation for transformers
No translations found for transformers
Finding translation for topas
No translations found for topas
Finding translation for tonino
No translations found for tonino
Finding tr

Finding translation for mojang
No translations found for mojang
Finding translation for miratorg
No translations found for miratorg
Finding translation for miradur
No translations found for miradur
Finding translation for mido
No translations found for mido
Finding translation for melaka
No translations found for melaka
Finding translation for mediform
No translations found for mediform
Finding translation for maslow
No translations found for maslow
Finding translation for masjid
No translations found for masjid
Finding translation for magda
No translations found for magda
Finding translation for mlc
No translations found for mlc
Finding translation for mfd
No translations found for mfd
Finding translation for mab
No translations found for mab
Finding translation for m760li
No translations found for m760li
Finding translation for m16
No translations found for m16
Finding translation for lống
No translations found for lống
Finding translation for luta
No translations found for luta
Find

Finding translation for conein
No translations found for conein
Finding translation for collège
No translations found for collège
Finding translation for collina
No translations found for collina
Finding translation for chronicles
No translations found for chronicles
Finding translation for chemistry
No translations found for chemistry
Finding translation for chanbanvanphongcomvn
No translations found for chanbanvanphongcomvn
Finding translation for ceylon
No translations found for ceylon
Finding translation for cessna
No translations found for cessna
Finding translation for caryn
No translations found for caryn
Finding translation for carsten
No translations found for carsten
Finding translation for carlmann
No translations found for carlmann
Finding translation for capacity
No translations found for capacity
Finding translation for calm
No translations found for calm
Finding translation for caffe
No translations found for caffe
Finding translation for cafef
No translations found for 

...

No translations found for muangười
Finding translation for minhgiải
No translations found for minhgiải
Finding translation for mims
No translations found for mims
Finding translation for micromollít
No translations found for micromollít
Finding translation for lướn
No translations found for lướn
Finding translation for lăngxê
No translations found for lăngxê
Finding translation for kên
No translations found for kên
Finding translation for kohinoorcomvn
No translations found for kohinoorcomvn
Finding translation for kigu
No translations found for kigu
Finding translation for kickoff
No translations found for kickoff
Finding translation for khị
No translations found for khị
Finding translation for khầu
No translations found for khầu
Finding translation for khướu
Finding translation for khôđể
No translations found for khôđể
Finding translation for khuyêt
No translations found for khuyêt
Finding translation for khoẳng
No translations found for khoẳng
Finding translation for khoằm
Finding t

No translations found for verza
Finding translation for veco
No translations found for veco
Finding translation for vbcc
No translations found for vbcc
Finding translation for vnand
No translations found for vnand
Finding translation for usborne
No translations found for usborne
Finding translation for uni5
No translations found for uni5
Finding translation for umass
No translations found for umass
Finding translation for ugc
No translations found for ugc
Finding translation for tốiquý
No translations found for tốiquý
Finding translation for tễu
No translations found for tễu
Finding translation for tunick
No translations found for tunick
Finding translation for tundra
No translations found for tundra
Finding translation for tulane
No translations found for tulane
Finding translation for tsing
No translations found for tsing
Finding translation for trunature
No translations found for trunature
Finding translation for trumpchi
No translations found for trumpchi
Finding translation for tr

Finding translation for pyotr
No translations found for pyotr
Finding translation for pylopass
No translations found for pylopass
Finding translation for purpose
No translations found for purpose
Finding translation for pulau
No translations found for pulau
Finding translation for propp
No translations found for propp
Finding translation for progress
No translations found for progress
Finding translation for producers
No translations found for producers
Finding translation for preventive
No translations found for preventive
Finding translation for premo
No translations found for premo
Finding translation for powershot
No translations found for powershot
Finding translation for potatso
No translations found for potatso
Finding translation for positano
No translations found for positano
Finding translation for poshanư
No translations found for poshanư
Finding translation for ponds
No translations found for ponds
Finding translation for poliva
No translations found for poliva
Finding tran

No translations found for l310
Finding translation for kym
No translations found for kym
Finding translation for kye
No translations found for kye
Finding translation for kuan
No translations found for kuan
Finding translation for kristeva
No translations found for kristeva
Finding translation for krakatau
No translations found for krakatau
Finding translation for koolaid
No translations found for koolaid
Finding translation for konjac
No translations found for konjac
Finding translation for kitsune
No translations found for kitsune
Finding translation for kingbbq
No translations found for kingbbq
Finding translation for kind
No translations found for kind
Finding translation for kimberley
No translations found for kimberley
Finding translation for kiki
No translations found for kiki
Finding translation for kidong
No translations found for kidong
Finding translation for kickboxing
No translations found for kickboxing
Finding translation for kejriwal
No translations found for kejriwal
F

IncompleteRead: IncompleteRead(14594 bytes read)

In [ ]:
#saving the batch
with open('./soha VI-EN batch {}.json'.format("34"), 'w') as outfile:
  json.dump(dictionary, outfile)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
dictionary

In [27]:
findTranslation("jba")

Finding translation for jba
No translations found for jba


'N/A'